In [ ]:
import numpy as np, sklearn as sk, pandas as pd
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import time as tm, os, re

%matplotlib inline

This is a preprocessing procedure that strips review of HTML tags and punctuation.

In [ ]:
def __wordlist( text, stops = None ) :
    letters_only = re.sub("[^a-zA-Z]", " ", bs( text ).get_text( ) )
    words = letters_only.lower( ).split()
    if stops is not None :
        return [ w for w in words if not w in stops ]
    return words

Convert the review text into a list of sentences, represented by lists of words.

In [ ]:
def __sentences( text, tokenizer = None, stops = None ):
    raw_sentences = tokenizer.tokenize( text.strip( ) )
    return [ __wordlist( s, stops = stops )
         for s in raw_sentences if len( s ) > 0 ]

Strip the review of all HTML tags and punctuation to get just a sequence of words.

In [ ]:
def clean_text( text, stops = None ) :
    words = __wordlist( text, stops = stops )
    return( " ".join( [ w for w in words ] ) )

Load NL data, necessary for preprocessing.

In [ ]:
from nltk.corpus import stopwords as nl_sw
import nltk.data

english_stopwords = set( nl_sw.words( "english" ) )
english_tokenizer = nltk.data.load( "tokenizers/punkt/english.pickle" )

Load the training data.

In [ ]:
train_data = pd.read_csv( './data/imdb/labeledTrainData.tsv',
    sep = "\t", header = 0, quoting = 3, encoding="utf-8" )

Clean the reviews.

In [ ]:
clean_train_reviews = [ clean_text( r, english_stopwords ) for r in train_data.review ]

Construct document features unsing the bag-of-words model.

In [ ]:
## Use the Bag of Words model to extract features.
from sklearn.feature_extraction.text import CountVectorizer

## SciKit vectorizer does exactly that: it constructs the set of most
##  frequently used words in the entire collection (fit), and then it
##  transforms each document into a frequency vector.
vectorizer = CountVectorizer( analyzer = "word", tokenizer = None,
    preprocessor = None, stop_words = None, max_features = 5000 )

## The returned feature matrix is sparse. 
train_data_features = vectorizer.fit_transform( clean_train_reviews )

Train a RandomForest ensemble classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
## Random forest constructs a tree classifiers on random (smaller) samples
##  of the data. This create a sort of bootstrapped distributio of trees
##  which are then used in ensemble, to make a classifier with less variance.
forest = RandomForestClassifier( n_estimators = 10, verbose = 0, max_depth = None )
forest = forest.fit( train_data_features, train_data.sentiment )

In [ ]:
predicted_train_sentiment = forest.predict( train_data_features )

np.where( predicted_train_sentiment != train_data.sentiment )

Test the classifier on the left-out unlabelled test sample.

In [ ]:
test_data = pd.read_csv( './data/imdb/testData.tsv',
    sep = "\t", header = 0, quoting = 3, encoding="utf-8" )

Clean the reviews, convert them to features and apply the random forest.

In [ ]:
clean_test_reviews = [ clean_text( r, english_stopwords ) for r in test_data.review ]
test_data_features = vectorizer.transform( clean_test_reviews )
predicted_test_sentiment = forest.predict( test_data_features )

Examine the results

In [ ]:
test_output = pd.DataFrame( data={ "id": test_data.id,
    "sentiment": predicted_test_sentiment, "review": test_data.review } )

In [ ]:
test_output

# Word 2 Vec

In [ ]:
import numpy as np, sklearn as sk, pandas as pd
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import time as tm, os, re

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

DATAPATH = os.path.realpath( os.path.join( ".", "data", "imdb" ) )

Load data for Natural Language processing.

In [ ]:
from nltk.corpus import stopwords as nl_sw
import nltk.data

english_stopwords = set( nl_sw.words( "english" ) )
english_tokenizer = nltk.data.load( "tokenizers/punkt/english.pickle" )

Load both labelled and unlabelled train datasets.

In [ ]:
# Read data from files 
unlabelled_train_data = pd.read_csv( os.path.join( DATAPATH, 'unlabeledTrainData.tsv' ), sep = "\t", header = 0, quoting = 3, encoding="utf-8" )
labelled_train_data = pd.read_csv( os.path.join( DATAPATH, 'labeledTrainData.tsv' ), sep = "\t", header = 0, quoting = 3, encoding="utf-8" )

Define preprocessors

In [ ]:
def __wordlist( text, stops = None ) :
    letters_only = re.sub("[^a-zA-Z]", " ", bs( text ).get_text( ) )
    words = letters_only.lower( ).split()
    if stops is not None :
        return [ w for w in words if not w in stops ]
    return words

Cut reviews into sentences.

In [ ]:
def __sentences( text, tokenizer = None, stops = None ):
    raw_sentences = tokenizer.tokenize( text.strip( ) )
    return [ __wordlist( s, stops = stops )
         for s in raw_sentences if len( s ) > 0 ]

Cut each review into sentences.

In [ ]:
train_sentences = list( )
if not os.path.exists( os.path.join( DATAPATH, 'imdb_review_train_sentences.txt' ) ) :
    print "Cutting reviews into sentences."
## Begin time
    tock = tm.time( )
## Convert reviews into sentences
    print "Labelled train dataset..."
    for r in labelled_train_data.review :
        train_sentences.extend( __sentences( r, english_tokenizer, stops = None ) )
    print "Unabelled train dataset..."
    for r in unlabelled_train_data.review :
        train_sentences.extend( __sentences( r, english_tokenizer, stops = None ) )
## End time
    tick = tm.time( )
## Report
    print "Preprocessing took %.1f sec." % ( tick - tock, )
    print "Caching..."
## Store the processed sentences in a UTF-8 text file
    with open( os.path.join( DATAPATH, 'imdb_review_train_sentences.txt' ), 'wb' ) as cache :
        cache.writelines( "\t".join( s ).encode( 'utf8' ) + "\n" for s in train_sentences )
## Final time
    tock = tm.time( )
else :
    print "Loading cached sentences..."
## Begin time
    tick = tm.time( )
    with open( os.path.join( DATAPATH, 'imdb_review_train_sentences.txt' ), 'rb' ) as cache :
        train_sentences.extend( l.decode( 'utf8' ).strip( ).split( '\t' ) for l in cache.readlines( ) )
## End time
    tock = tm.time( )
## Report
print "Loaded sentences in %.1f sec." % ( tock - tick, )

Get the vectore representation of words using word2vec in gensim module.

In [ ]:
import gensim.models, time as tm
# Initialize the model
model = gensim.models.Word2Vec(
    workers = 4,           # Number of threads to run in parallel
    size = 400,            # Word vector dimensionality
    min_count = 30,        # Minimum word count for pruning the internal dictionary
    window = 12,           # Context sindow size
    sample = 1e-3 )        # Downsample setting for frequent words

model_cache_name = "W2V_%d-%d-%d.mdl" % ( model.layer1_size, model.min_count, model.window , )
if not os.path.exists( os.path.join( DATAPATH, model_cache_name ) ) :
## Begin time
    tock = tm.time( )
## First pass -- building the vocabulary
    model.build_vocab( train_sentences )
## Second pass --  training the neural net
    model.train( train_sentences )
## End time
    tick = tm.time( )
## Report
    print "Training word2vec took %.1f sec." % ( tick - tock, )
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
    model.init_sims( replace = True )
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
    model.save( os.path.join( DATAPATH, model_cache_name ) )
## End time
    tock = tm.time( )
else :
## Begin time
    tick = tm.time( )
## Load the model from the blob
    model = gensim.models.Word2Vec.load( os.path.join( DATAPATH, model_cache_name ) )
## End time
    tock = tm.time( )
## Report
print "Model loaded in %.1f sec." % ( tock - tick, )

Let's see how well the trained model performs over the google analogical proportions dataset.

In [ ]:
print "Testing Google's analogical proportions..."
tick = tm.time( )
## test model accuracy against the Google dataset
google_dataset_accuracy = model.accuracy( os.path.join( DATAPATH, 'questions-words.txt' ) )
tock = tm.time( )
print "Completed in %.1f sec." % ( tock - tick, )

In [ ]:
print "CORRECT\tTOTAL\tSECTION"
for s in google_dataset_accuracy :
    total = len( s['correct'] ) + len( s['incorrect'] )
    print "%4d\t%5d\t%s." % ( len( s['correct'] ), total, s['section'], )

The reason why thiese results are so poor is that the reviews database is not a language corpus, it is does not provide enough coverage of the natural language variety (English), it is topically biased, and, since it is mainly user generated content, it is stylistically more colloquial.

In [ ]:
print model.most_similar( "queen" )

In [ ]:
print model.most_similar( positive=['woman', 'king'], negative=['man'], topn=1)
print model.doesnt_match("breakfast cereal dinner lunch".split())
print model.similarity('woman', 'man')

In [ ]:
vocab = np.asarray( model.vocab.keys(), dtype = np.str)
# vocab[ np.argmax( np.abs(model.syn0), axis = 0 ) ]

In [ ]:
vocab